In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('Twitter_Data.csv')
df2 = pd.read_csv('Reddit_Data.csv')

In [3]:
print(df1.shape)
print(df2.shape)

(162980, 2)
(37249, 2)


In [4]:
print('twitter sentiment:\n')
print(df1.head(5))
print('\n\nreddit sentiment:\n')
print(df2.head(5))

twitter sentiment:

                                          clean_text  category
0  when modi promised “minimum government maximum...      -1.0
1  talk all the nonsense and continue all the dra...       0.0
2  what did just say vote for modi  welcome bjp t...       1.0
3  asking his supporters prefix chowkidar their n...       1.0
4  answer who among these the most powerful world...       1.0


reddit sentiment:

                                       clean_comment  category
0   family mormon have never tried explain them t...         1
1  buddhism has very much lot compatible with chr...         1
2  seriously don say thing first all they won get...        -1
3  what you have learned yours and only yours wha...         0
4  for your own benefit you may want read living ...         1


In [5]:
df1.loc[0]['clean_text']

'when modi promised “minimum government maximum governance” expected him begin the difficult job reforming the state why does take years get justice state should and not business and should exit psus and temples'

In [6]:
print('Twitter NULL:\n')
print(df1.isnull().sum())
print('\n\n\nReddit NULL:\n')
print(df2.isnull().sum())

Twitter NULL:

clean_text    4
category      7
dtype: int64



Reddit NULL:

clean_comment    100
category           0
dtype: int64


In [7]:
df1.columns

Index(['clean_text', 'category'], dtype='object')

In [8]:
df2.columns

Index(['clean_comment', 'category'], dtype='object')

In [9]:
df1.dropna(inplace=True)
df2.dropna(inplace=True)

In [10]:
print('Twitter NULL:\n')
print(df1.isnull().sum())
print('\n\n\nReddit NULL:\n')
print(df2.isnull().sum())

Twitter NULL:

clean_text    0
category      0
dtype: int64



Reddit NULL:

clean_comment    0
category         0
dtype: int64


In [11]:
df1.describe()

,category
count,162969.000000
mean,0.225442
std,0.781279
min,-1.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [12]:
df2.describe()

,category
count,37149.000000
mean,0.203316
std,0.779491
min,-1.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [20]:
df2['clean_text'] = df2['clean_comment']

In [21]:
df2.head()

,clean_comment,category,clean_text
0,family mormon have never tried explain them t...,1,family mormon have never tried explain them t...
1,buddhism has very much lot compatible with chr...,1,buddhism has very much lot compatible with chr...
2,seriously don say thing first all they won get...,-1,seriously don say thing first all they won get...
3,what you have learned yours and only yours wha...,0,what you have learned yours and only yours wha...
4,for your own benefit you may want read living ...,1,for your own benefit you may want read living ...


In [22]:
df2 = df2.drop(['clean_comment'],axis=1)

In [23]:
df = df1.append(df2)

In [24]:
df.shape

(200118, 2)

In [25]:
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
